## Data Ingestion

In [1]:
## Document datastructure

from langchain_core.documents import Document

In [2]:
doc = Document(
    page_content="Main file content",
    metadata=(
        {
            "Source": "example.text",
            "pages": 1,
            "author": "Aquib"
        }
    )
)

In [3]:
doc

Document(metadata={'Source': 'example.text', 'pages': 1, 'author': 'Aquib'}, page_content='Main file content')

In [4]:
## Text Loader

from langchain_community.document_loaders import TextLoader

loader = TextLoader(r"..\data\text_files\python.txt",encoding='utf-8')
document = loader.load()
print(document)

c:\Users\maquib\projects\rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'source': '..\\data\\text_files\\python.txt'}, page_content='Python Programming Language\n\nPython is a high-level, interpreted programming language renowned for its elegant syntax and exceptional readability. Created by Guido van Rossum and first released in 1991, Python has evolved into one of the world\'s most popular and versatile programming languages.\n\nCore Features\n1. Simple and Readable Syntax: Uses indentation for code blocks and emphasizes code readability\n2. Dynamic Typing: Variables are dynamically typed, making development more flexible\n3. Rich Standard Library: Comes with "batteries included" for diverse programming tasks\n4. Cross-Platform: Runs on Windows, macOS, Linux, and other platforms\n5. Active Community: Large, supportive community creating numerous third-party packages\n\nKey Applications\n- Web Development: Frameworks like Django and Flask\n- Data Science: NumPy, Pandas, and SciPy libraries\n- Machine Learning: TensorFlow, PyTorch, and 

In [5]:
## Directory Loader
from langchain_community.document_loaders import DirectoryLoader

dir_loader = DirectoryLoader(
    r"..\data\text_files",
    glob = "**/*.txt",
    loader_cls= TextLoader, # if there is pdf also then use list of loaders
    loader_kwargs={'encoding':'utf-8'},
    show_progress=True
)

dir_document = dir_loader.load()
print(dir_document)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 60.94it/s]

[Document(metadata={'source': '..\\data\\text_files\\ml.txt'}, page_content='Machine Learning - overview and notes\n\nWhat is ML:\nMachine learning (ML) builds models that learn patterns from data to make predictions or decisions without being explicitly programmed.\n\nMain types:\n- Supervised learning: regression, classification (labels provided).\n- Unsupervised learning: clustering, dimensionality reduction (no labels).\n- Reinforcement learning: agents learn via rewards and interactions.\n\nTypical workflow:\n1. Problem definition and metrics.\n2. Data collection and exploration (EDA).\n3. Preprocessing: cleaning, imputation, scaling, categorical encoding.\n4. Feature engineering and selection.\n5. Train/validation/test split; cross-validation.\n6. Model training and hyperparameter tuning.\n7. Evaluation on hold-out test set.\n8. Deployment and monitoring.\n\nCommon algorithms:\n- Linear models: linear regression, logistic regression.\n- Tree-based: decision trees, random forest, 

In [6]:
# for pdf
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader

pdf_loader = DirectoryLoader(
    r"..\data\pdf_files",
    glob = "**/*.pdf",
    loader_cls= PyMuPDFLoader,
    show_progress=True
)

pdf_document = pdf_loader.load()
pdf_document

100%|██████████| 2/2 [00:01<00:00,  1.18it/s]


[Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20251023172502', 'source': '..\\data\\pdf_files\\attention.pdf', 'file_path': '..\\data\\pdf_files\\attention.pdf', 'total_pages': 11, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': 'D:20251023172502', 'page': 0}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. T

| Feature          | **PyPDFLoader**    | **PyMuPDFLoader**              |
| ---------------- | ------------------ | ------------------------------ |
| Library          | PyPDF2             | PyMuPDF (fitz)                 |
| Speed            | ⚡ Fast             | 🐢 Slightly slower             |
| Text Extraction  | Basic (plain text) | Advanced (preserves structure) |
| Layout / Columns | ❌ Ignored          | ✅ Preserved                    |
| Scanned PDFs     | ❌ No               | ⚠️ Partial (with OCR)          |
| Best For         | Simple text PDFs   | Complex or formatted PDFs      |


DirectoryLoader uses one loader_cls for all files, so it can’t automatically choose the correct loader per extension

LangChain’s DirectoryLoader allows a custom callable for loader_cls — meaning you can define a function that returns the correct loader based on file extension.

In [8]:
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader, TextLoader, CSVLoader

def dynamic_loader(file_path):
    """Return the right loader depending on file extension."""
    if file_path.lower().endswith(".pdf"):
        return PyMuPDFLoader(file_path)
    elif file_path.lower().endswith(".txt"):
        return TextLoader(file_path, encoding="utf-8")
    elif file_path.lower().endswith(".csv"):
        return CSVLoader(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_path}")

folder_path = r"..\data"

# DirectoryLoader can now handle mixed formats
mixed_loader = DirectoryLoader(
    folder_path,
    glob="**/*.*",
    loader_cls=dynamic_loader,   # <— dynamic decision
    show_progress=True
)

all_docs = mixed_loader.load()

print(f"✅ Loaded {len(all_docs)} documents from mixed formats.")


100%|██████████| 4/4 [00:00<00:00, 37.49it/s]

✅ Loaded 23 documents from mixed formats.


## RAG Pipeline - Data Ingestion to vectorDB PipeLine

In [1]:
import os
from langchain_community.document_loaders import TextLoader,DirectoryLoader,PyMuPDFLoader,PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path


c:\Users\maquib\projects\rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Read all the file inside the dir
def process_all_pdfs(pdf_directory):
    all_documents = []
    pdf_dir = Path(pdf_directory)

    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(pdf_file)
            documents = loader.load()

            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = "pdf"
            
            all_documents.extend(documents)
            print(f" Loaded {len(documents)} pages")
        except Exception as e:
            print(f"Error: {e}")
    print(f"\nTotal Documents loaded: {len(all_documents)}")
    return all_documents

all_pdf_documents = process_all_pdfs("../data")

Found 2 PDF files to process

Processing: attention.pdf
 Loaded 11 pages

Processing: yolo.pdf
 Loaded 10 pages

Total Documents loaded: 21


In [4]:
## For Production purpose
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader, TextLoader, CSVLoader
from pathlib import Path

def process_all_documents(data_directory: str):
    """
    Use DirectoryLoader to load all PDFs, TXTs, and CSVs from one directory.
    Automatically applies the correct loader per file type.
    """
    
    # Define a dynamic loader selector
    def dynamic_loader(file_path: str):
        if file_path.lower().endswith(".pdf"):
            return PyMuPDFLoader(file_path)
        elif file_path.lower().endswith(".txt"):
            return TextLoader(file_path, encoding="utf-8")
        elif file_path.lower().endswith(".csv"):
            return CSVLoader(file_path)
        else:
            raise ValueError(f"❌ Unsupported file type: {file_path}")

    # Create a single DirectoryLoader that handles all formats
    loader = DirectoryLoader(
        data_directory,
        glob="**/*.*",          # Include all file types
        loader_cls=dynamic_loader,
        show_progress=True
    )

    print("📂 Scanning directory for supported files...")
    all_documents = loader.load()
    print(f"✅ Total documents loaded: {len(all_documents)}")

    # Add metadata (file name + type)
    for doc in all_documents:
        path = Path(doc.metadata["source"])
        doc.metadata["source_file"] = path.name
        doc.metadata["file_type"] = path.suffix.lower().replace(".", "")

    return all_documents


# --- Run it ---
all_docs = process_all_documents("../data")
print(f"📊 Final count: {len(all_docs)} documents loaded.")


📂 Scanning directory for supported files...


100%|██████████| 4/4 [00:00<00:00, 27.61it/s]

✅ Total documents loaded: 23
📊 Final count: 23 documents loaded.


In [5]:
### Text Splitting

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap,
        length_function = len,
        separators= ["\n\n","\n"," ",""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    if split_docs:
        print("\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    return split_docs

In [6]:
chunks = split_documents(all_pdf_documents)

Split 21 documents into 98 chunks

Example chunk:
Content: Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz...
Metadata: {'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20251023172502', 'source': '..\\data\\pdf_files\\attention.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1', 'source_file': 'attention.pdf', 'file_type': 'pdf'}


| Use Case                                  | Recommended `chunk_size` | Recommended `chunk_overlap` | Notes                             |
| ----------------------------------------- | ------------------------ | --------------------------- | --------------------------------- |
| 🧾 Short text files (e.g., emails, notes) | 500–1000                 | 50–100                      | Preserve small context            |
| 📚 Long PDFs or articles                  | 1000–2000                | 150–300                     | Balance context + performance     |
| 🧠 LLM retrieval (RAG)                    | 500–1500                 | 100–200                     | Avoid context cutoff mid-sentence |
| ⚙️ Code or structured data                | 200–500                  | 50–100                      | Finer-grained chunks help search  |
| 🪶 Summarization (few-shot context)       | 2000–4000                | 200–400                     | Larger chunks preserve flow       |


⚖️ General Rules of Thumb

Overlap ≈ 10–20% of chunk size → keeps coherence without too much duplication.

Smaller chunks → better for search and retrieval.

Larger chunks → better for summarization or context-rich tasks.

Tune based on embedding model’s context window (e.g., 1024–8192 tokens).


### Embedding and VectorStore DB

In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity


In [18]:
# class EmbeddingManager:
#     def __init__(self,model_name:str = "all-MiniLM-L6-v2"):
#         self.model_name = model_name
#         self.model = None
#         self._load_model()
    
#     def _load_model(self):
#         try:
#             print(f"Loading Embedding Model: {self.model_name}")
#             self.model = SentenceTransformer(self.model_name)
#             print(f"Model Loaded Successfully. Embedding Dimension: {self.model.get_sentence_embedding_dimension()}")
#         except Exception as e:
#             raise RuntimeError(f"Failed to load model '{self.model_name}': {e}")

    
#     def generate_embeddings(self,texts:List[str]) -> np.ndarray:
#         if not self.model:
#             raise ValueError("Model not loaded")
#         print(f"Generating embeddings for {len(texts)} texts...")
#         if isinstance(texts, str):
#             texts = [texts]
#         embeddings = self.model.encode(texts,show_progress_bar=True)
#         print(f"Generated embeddings with shape: {embeddings.shape}")
#         return embeddings
    

# For Azure Openai Embeddings

import os
import numpy as np
from typing import List
from openai import AzureOpenAI

class EmbeddingManager:
    def __init__(self, model_name: str = None):
        # Read environment variables
        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
        api_key = os.getenv("AZURE_OPENAI_API_KEY")
        api_version = os.getenv("AZURE_OPENAI_API_VERSION")
        
        if not azure_endpoint or not api_key or not api_version:
            raise ValueError("AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_API_KEY, and AZURE_OPENAI_API_VERSION must all be set in environment variables.")
        
        # Default to your deployed model name if not passed
        self.model_name = model_name or os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")

        # Initialize Azure OpenAI client
        self.client = AzureOpenAI(
            api_key=api_key,
            api_version=api_version,
            azure_endpoint=azure_endpoint
        )

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        if not isinstance(texts, list) or not all(isinstance(t, str) for t in texts):
            raise TypeError("Input must be a list of strings.")

        print(f"Generating embeddings for {len(texts)} texts using model: {self.model_name}")

        response = self.client.embeddings.create(
            model=self.model_name,
            input=texts
        )

        embeddings = np.array([item.embedding for item in response.data], dtype=np.float32)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


In [ ]:
embedding_manager = EmbeddingManager(model_name="text-embedding-3-large")

In [20]:
## VectorStore

class VectorStore:
    def __init__(self,collection_name:str="pdf_documents",persist_directory:str="../data/vector_store"):

        self.collection_name =  collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()
    
    def _initialize_store(self):
        try:
            os.makedirs(self.persist_directory,exist_ok=True)
            self.client =  chromadb.PersistentClient(path=self.persist_directory)

            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata= {"description": "PDF document embeddings for RAG"}
            )
            print(f"vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise
    
    def add_documents(self,documents:List[Any],embeddings:np.ndarray):
        if len(documents)!=len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        print(f"Adding {len(documents)} documents to vector store...")

        #prepare data for chromadb
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i ,(doc,embedding) in enumerate(zip(documents,embeddings)):
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            metadata = metadata = dict(doc.metadata) if getattr(doc, "metadata", None) else {}
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            documents_text.append(doc.page_content)
            embeddings_list.append(embedding.tolist())

        try:
            self.collection.add(
                ids = ids,
                embeddings = embeddings_list,
                metadatas = metadatas,
                documents= documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in the collections: {self.collection.count()}")
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise


In [21]:
vectorstore = VectorStore()

vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [22]:
texts = [doc.page_content for doc in chunks]
embeddings = embedding_manager.generate_embeddings(texts)

vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 98 texts using model: text-embedding-3-large
Generated embeddings with shape: (98, 3072)
Adding 98 documents to vector store...
Successfully added 98 documents to vector store
Total documents in the collections: 98


## Retriever Pipeline From VectorStore

In [23]:
class RAGRetriever:
    def __init__(self,vector_store:VectorStore, embedding_manager: EmbeddingManager):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager
    
    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []


In [24]:
rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [27]:
rag_retriever.retrieve("What is yolo model")

Retrieving documents for query: 'What is yolo model'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts using model: text-embedding-3-large
Generated embeddings with shape: (1, 3072)
Retrieved 2 documents (after filtering)


[{'id': 'doc_29f0ace4_44',
  'content': 'YOLO model processes images in real-time at 45 frames\nper second. A smaller version of the network, Fast YOLO,\nprocesses an astounding 155 frames per second while\nstill achieving double the mAP of other real-time detec-\ntors. Compared to state-of-the-art detection systems, YOLO\nmakes more localization errors but is less likely to predict\nfalse positives on background. Finally, YOLO learns very\ngeneral representations of objects. It outperforms other de-\ntection methods, including DPM and R-CNN, when gener-\nalizing from natural images to other domains like artwork.\n1. Introduction\nHumans glance at an image and instantly know what ob-\njects are in the image, where they are, and how they inter-\nact. The human visual system is fast and accurate, allow-\ning us to perform complex tasks like driving with little con-\nscious thought. Fast, accurate algorithms for object detec-\ntion would allow computers to drive cars without special-\nize

In [23]:
"""import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
model_name = "gpt-4o"
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

subscription_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "hello how are you?",
        }
    ],
    stream=True,
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

# print(response.choices[0].message.content)

for update in response:
    if update.choices:
        print(update.choices[0].delta.content or "", end="")

client.close()"""

'import os\nfrom openai import AzureOpenAI\nfrom dotenv import load_dotenv\n\nload_dotenv(override=True)\n\nendpoint = os.getenv("AZURE_OPENAI_ENDPOINT")\nmodel_name = "gpt-4o"\ndeployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")\n\nsubscription_key = os.getenv("AZURE_OPENAI_API_KEY")\napi_version = os.getenv("AZURE_OPENAI_API_VERSION")\n\nclient = AzureOpenAI(\n    api_version=api_version,\n    azure_endpoint=endpoint,\n    api_key=subscription_key,\n)\n\nresponse = client.chat.completions.create(\n    messages=[\n        {\n            "role": "system",\n            "content": "You are a helpful assistant.",\n        },\n        {\n            "role": "user",\n            "content": "hello how are you?",\n        }\n    ],\n    stream=True,\n    max_tokens=4096,\n    temperature=1.0,\n    top_p=1.0,\n    model=deployment\n)\n\n# print(response.choices[0].message.content)\n\nfor update in response:\n    if update.choices:\n        print(update.choices[0].delta.content or "", end=""

## RAG Pipeline- VectorDB To LLM Output Generation

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage,SystemMessage 

llm = AzureChatOpenAI(
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=0.7
)

In [29]:
def rag_simple(query,retriever,llm,top_k=3):
    results = retriever.retrieve(query,top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question"
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response = llm.invoke([
        SystemMessage(content="You are a helpful assistant that uses provided context."),
        HumanMessage(content=prompt)
        ])
    return response.content
    
answer=rag_simple("model architecture of attention model",rag_retriever,llm)
print(answer)


Retrieving documents for query: 'model architecture of attention model'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts using model: text-embedding-3-large
Generated embeddings with shape: (1, 3072)
Retrieved 3 documents (after filtering)
The attention model architecture, specifically the Transformer, consists of an encoder-decoder structure. The encoder stack maps an input sequence to continuous representations using stacked self-attention and point-wise, fully connected layers. The decoder stack generates the output sequence auto-regressively, using previously generated symbols as input. Each layer in both the encoder and decoder employs residual connections followed by layer normalization, with all sub-layers producing outputs of dimension **dmodel = 512**. The decoder includes an additional sub-layer for multi-head attention over the encoder outputs and modifies the self-attention sub-layer to mask future positions, ensuring predictions depend only on known prior o

In [30]:
## Enhance the RAG Pipeline

def rag_advance(query,retriever,llm,top_k=5,min_score=0.2,return_context = False):
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        "source": doc["metadata"].get("source_file",doc["metadata"].get("source","unknown")),
        "page": doc["metadata"].get("page","unknown"),
        "score": doc['similarity_score'],
        "preview": doc["content"][:300] + "..."
    }for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])

    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])

    output = {
        'answer': response.content,
        "sources": sources,
        "confidence": confidence
    }
    if return_context:
        output['context'] = context
    return output


In [31]:
result = rag_advance("dimension of embeddings in attention model", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'dimension of embeddings in attention model'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts using model: text-embedding-3-large
Generated embeddings with shape: (1, 3072)
Retrieved 3 documents (after filtering)
Answer: The dimension of embeddings in the attention model is **dmodel**.
Sources: [{'source': 'attention.pdf', 'page': 2, 'score': 0.23711854219436646, 'preview': 'around each of the sub-layers, followed by layer normalization. We also modify the self-attention\nsub-layer in the decoder stack to prevent positions from attending to subsequent positions. This\nmasking, combined with fact that the output embeddings are offset by one position, ensures that the\npredi...'}, {'source': 'attention.pdf', 'page': 5, 'score': 0.2141849398612976, 'preview': 'Table 1: Maximum path lengths, per-layer complexity and minimum number of sequential operations\nfor different layer types. nis the sequence length, dis the representation dimension, k

### Advanced RAG Pipeline: Streaming, Citations, History, Summarization

In [43]:
from typing import Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]

            prompt = f"""Use the following context to answer the question concisely.
Context:
{context}

Question: {question}

Answer:"""

            if stream:
                print("Streaming answer (simulated):")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()

            response = self.llm.invoke([prompt])
            answer = getattr(response, 'content', response)

        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + ("\n\nCitations:\n" + "\n".join(citations) if citations else "")

        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = getattr(summary_resp, 'content', summary_resp)

        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }


In [44]:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("architecture of yolo model", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'architecture of yolo model'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts using model: text-embedding-3-large
Generated embeddings with shape: (1, 3072)
Retrieved 1 documents (after filtering)
Streaming answer (simulated):
Use the following context to answer the question concisely.
Context:
Our network architecture is inspired by the GoogLeNet
model for image classiﬁcation [34]. Our network has 24
convolutional layers followed by 2 fully connected layers.
Instead of the inception modules used by GoogLeNet, we
simply use 1 ×1 reduction layers followed by 3 ×3 convo-
lutional layers, similar to Lin et al [22]. The full network is
shown in Figure 3.
We also train a fast version of YOLO designed to push
the boundaries of fast object detection. Fast YOLO uses a
neural network with fewer convolutional layers (9 instead
of 24) and fewer ﬁlters in those layers. Other than the size
of the network, all training and testing parameters are the
sa

In [49]:
## with streaming

from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage,SystemMessage 

llm = AzureChatOpenAI(
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=0.7,streaming=True
)
from typing import Dict, Any
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage

class AdvancedRAGPipelineStream:
    def __init__(self, retriever, llm: AzureChatOpenAI):
        self.retriever = retriever
        self.llm = llm
        self.history = []

    def query(
        self,
        question: str,
        top_k: int = 5,
        min_score: float = 0.2,
        stream: bool = False,
        summarize: bool = False
    ) -> Dict[str, Any]:
        # 1. Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)

        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]

            prompt = f"""Use the following context to answer the question concisely.
Context:
{context}

Question: {question}

Answer:"""

            # 2. Generate answer
            if stream:
                # Streaming mode
                print("Streaming answer:")
                answer = ""
                for chunk in self.llm.stream([HumanMessage(content=prompt)]):
                    token = getattr(chunk, "content", str(chunk))
                    answer += token
                    print(token, end='', flush=True)
                print()  # newline after streaming
            else:
                # Non-streaming mode
                response = self.llm.generate([[HumanMessage(content=prompt)]])
                answer = response.generations[0][0].text

        # 3. Add citations
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + ("\n\nCitations:\n" + "\n".join(citations) if citations else "")

        # 4. Optional summary
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.generate([[HumanMessage(content=summary_prompt)]])
            summary = summary_resp.generations[0][0].text

        # 5. Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        # 6. Return results
        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }


    
adv_rag = AdvancedRAGPipelineStream(rag_retriever, llm)
result = adv_rag.query("architecture of yolo model", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])


Retrieving documents for query: 'architecture of yolo model'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts using model: text-embedding-3-large
Generated embeddings with shape: (1, 3072)
Retrieved 1 documents (after filtering)
Streaming answer:
The YOLO model architecture is inspired by the GoogLeNet model for image classification. It consists of 24 convolutional layers followed by 2 fully connected layers. Instead of using inception modules like GoogLeNet, YOLO employs 1×1 reduction layers followed by 3×3 convolutional layers. A faster version of the model, called Fast YOLO, has a smaller architecture with 9 convolutional layers and fewer filters while maintaining the same training and testing parameters.

Final Answer: The YOLO model architecture is inspired by the GoogLeNet model for image classification. It consists of 24 convolutional layers followed by 2 fully connected layers. Instead of using inception modules like GoogLeNet, YOLO employs 1×1 reduction layers 